# 워크넷 API 이용 기업 정보 데이터 베이스화 

간단 요약 설명 : 워크넷 open api 를 이용해서 채용정보를 얻어 오려고 한다. 
채용정보직종은 데이터 분석과 관련된 직종이며 상세하게 얻어오려고 한다. 
파이썬으로는 얻어온 정보를 정리하고 + 카카오api를 이용해 좌표까지 얻어와서 
csv파일로 만드는 것까지가 목표( 시각화를 위한 tableau 활용 예정) $$ 

1. 모듈 설치

In [144]:
import requests     # url 이용해서 정보 얻기 
import pandas as pd # pandas 데이터프레임화해서 자료정리 
import numpy as np
from bs4 import BeautifulSoup # 뷰티풀숲으로 자료 원하는 것만 추출 



2. 검색할 직업 목록 확인 (데이터 관련직만 확인해보자)

In [17]:
code = pd.read_csv('/Users/snoo/enterprise/직종코드1.csv',encoding='utf-8',
                   index_col='Unnamed: 0') 
code.columns = ['직종코드','대분류','중분류','소분류']
code

,직종코드,대분류,중분류,소분류
0,NaN,NaN,NaN,NaN
1,카테고리 ID,1 depth,2 depth,3_depth
2,01,경영·사무·금융·보험,,
3,011,,행정·경영·금융·보험 관리직,
4,011100,,,의회의원·고위공무원 및 공공단체임원
...,...,...,...,...
1294,135,,농림어업 단순 종사자,
1295,905000,,,농림어업 단순 종사원
1296,905001,,,농업 단순 종사원
1297,905002,,,"임업 단순 종사원(산림보호감시, 산불감시원 등)"


2. 1) 부분 문자열 검사를 통해 직종 코드 찾기 

In [28]:
code[code['소분류'].str.contains('데이터',na=False)] 

,직종코드,대분류,중분류,소분류
249,134100,,,데이터 전문가
250,134101,,,데이터 설계 및 프로그래머
251,134102,,,데이터 분석가(빅데이터 분석가)
252,134103,,,데이터베이스 운영·관리자
258,134900,,,디지털 포렌식 전문가 및 기타 데이터·네트워크 전문가


In [46]:
# db 관리자나 디지털포렌식 전문가는 원하는 직종이 아니기 때문에 제외하도록 하자
need_code = code['직종코드'][code['소분류'].str.contains('데이터',na=False)]
need_code = need_code[:3]

3. 워크넷 open api 사용법 
    - 워크넷 오픈api 사이트 회원가입 필수 (인증키 신청) 
    - 인증키 get 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=(인증키)&callTp=L&returnType=XML&startPage=1&display=100&occupation=134100'

인증키 = 인증키 
callTp = 얻을 정보 기본 L, 상세 = D 상세는 밑에 다룰 예정 
returnType = XML 
startPage = 시작페이지 
display = 최대 100
occupation = 직종코드
    



In [69]:
need_code.index = range(0,3)
need_code[0]

'134100'

In [82]:
# 먼저 하나만 먼저 해보도록한다
key = 'WNL596OLQCP7023TCKJPB2VR1HJ'

url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?\
authKey='+key+'&callTp=L&returnType=XML&startPage=1&\
display=100&occupation='+need_code[0]
print(url)

http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=WNL596OLQCP7023TCKJPB2VR1HJ&callTp=L&returnType=XML&startPage=1&display=100&occupation=134100


4. request 모듈을 사용해서 정보 얻기

In [83]:
res = requests.get(url)
text = res.text
text 
# 94 + 34 + 28 = 156

'<?xml version="1.0" encoding="UTF-8"?><wantedRoot><total>94</total><startPage>1</startPage><display>94</display><wanted><wantedAuthNo>K151632207080067</wantedAuthNo><company>주식회사디엠시스템엔지니어링</company><title>(경력) 데이터베이스 개발 및 어플리케이션 유지관...</title><salTpNm>월급</salTpNm><sal>250만원 ~ 250만원</sal><minSal>2500000</minSal><maxSal>2500000</maxSal><region>경기 성남시 중원구</region><holidayTpNm>주5일근무</holidayTpNm><minEdubg>학력무관</minEdubg><career>경력</career><regDt>22-07-08</regDt><closeDt>채용시까지  22-09-07</closeDt><infoSvc>VALIDATION</infoSvc><wantedInfoUrl>http://www.work.go.kr/empDetailRedirect.do?wantedAuthNo=K151632207080067</wantedInfoUrl><wantedMobileInfoUrl>https://m.work.go.kr/regionJobsWorknet/jobDetailView2.do?srchInfotypeNm=VALIDATION&amp;srchWantedAuthNo=K151632207080067</wantedMobileInfoUrl><smodifyDtm>202207081615</smodifyDtm><zipCd>13174</zipCd><strtnmCd>411333179004</strtnmCd><basicAddr>경기도 성남시 중원구 광명로 377</basicAddr><detailAddr>신구대학 창업관 103호</detailAddr><empTpCd>11</empTpCd><jobsCd>133207</j

In [33]:
# 뷰티풀숲의 find_all 같은 기능으로 원하는 정보만 얻을 생각이다.
soup = BeautifulSoup(text,'lxml-xml')
soup
# display = 94 총 94개의 구직 정보가 있음 

<?xml version="1.0" encoding="utf-8"?>
<wantedRoot><total>94</total><startPage>1</startPage><display>94</display><wanted><wantedAuthNo>K151632207080067</wantedAuthNo><company>주식회사디엠시스템엔지니어링</company><title>(경력) 데이터베이스 개발 및 어플리케이션 유지관...</title><salTpNm>월급</salTpNm><sal>250만원 ~ 250만원</sal><minSal>2500000</minSal><maxSal>2500000</maxSal><region>경기 성남시 중원구</region><holidayTpNm>주5일근무</holidayTpNm><minEdubg>학력무관</minEdubg><career>경력</career><regDt>22-07-08</regDt><closeDt>채용시까지  22-09-07</closeDt><infoSvc>VALIDATION</infoSvc><wantedInfoUrl>http://www.work.go.kr/empDetailRedirect.do?wantedAuthNo=K151632207080067</wantedInfoUrl><wantedMobileInfoUrl>https://m.work.go.kr/regionJobsWorknet/jobDetailView2.do?srchInfotypeNm=VALIDATION&amp;srchWantedAuthNo=K151632207080067</wantedMobileInfoUrl><smodifyDtm>202207081615</smodifyDtm><zipCd>13174</zipCd><strtnmCd>411333179004</strtnmCd><basicAddr>경기도 성남시 중원구 광명로 377</basicAddr><detailAddr>신구대학 창업관 103호</detailAddr><empTpCd>11</empTpCd><jobsCd>133207</j

5. 뷰티풀숲을 이용해 구인인증번호('wantedAuthNo')를 얻을 생각이다. 
 why? / 상세를 보기 위해서는 구인인증번호가 필요하다. 

In [40]:
# 전체의 구인인증번호를 반복문을 통해서 리스트로 만들 생각이다. 
soup.find_all('wantedAuthNo')[0].text 

'K151632207080067'

In [43]:
guin_num = []
for i in range(int(soup.find('display').text)):
    guin_num.append(soup.find_all('wantedAuthNo')[i].text)
guin_num

['K151632207080067',
 'K131112207080031',
 'K131212207080047',
 'K140022207070078',
 'K151322207070004',
 'K151132207070018',
 'K170052207060069',
 'K131412207060036',
 'K151812207060043',
 'K120412207060035',
 'K170052207060050',
 'K120572207060007',
 'K120312207050062',
 'K140062207050060',
 'K151342207050062',
 'K120252207050033',
 'K151342207050042',
 'K120612207050060',
 'K130042207050013',
 'K151152207040280',
 'K120312207040044',
 'K151712207040013',
 'K151712207040012',
 'K163012207010071',
 'K120612207010071',
 'K120422207010066',
 'K120422207010040',
 'K170052207010025',
 'K151222207010050',
 'KJEA002207010001',
 'K131332206290039',
 'K172122206290032',
 'K131132206280059',
 'K120032206280044',
 'K151412206280064',
 'K140132206270087',
 'K170052206270074',
 'K151572206270056',
 'K170052206270048',
 'K160042206270016',
 'K162142206270009',
 'K120142206230051',
 'K120612206230051',
 'K151122206220105',
 'K170052206220007',
 'K151642206210010',
 'K140022206200117',
 'K1600122061

## 직종코드 3가지를 url에 넣고 구인인증번호를 하나의 리스트로 저장하자 

In [76]:
key = 'WNL596OLQCP7023TCKJPB2VR1HJ'
guin_num_all = []

for i in range(len(need_code)):
    url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?\
authKey='+key+'&callTp=L&returnType=XML&startPage=1&\
display=100&occupation='+need_code[i]
    res = requests.get(url)
    text = res.text
    soup = BeautifulSoup(text,'lxml-xml')
    for j in range(int(soup.find('display').text)):
        guin_num_all.append(soup.find_all('wantedAuthNo')[j].text)

guin_num_all



['K151632207080067',
 'K131112207080031',
 'K131212207080047',
 'K140022207070078',
 'K151322207070004',
 'K151132207070018',
 'K170052207060069',
 'K131412207060036',
 'K151812207060043',
 'K120412207060035',
 'K170052207060050',
 'K120572207060007',
 'K120312207050062',
 'K140062207050060',
 'K151342207050062',
 'K120252207050033',
 'K151342207050042',
 'K120612207050060',
 'K130042207050013',
 'K151152207040280',
 'K120312207040044',
 'K151712207040013',
 'K151712207040012',
 'K163012207010071',
 'K120612207010071',
 'K120422207010066',
 'K120422207010040',
 'K170052207010025',
 'K151222207010050',
 'KJEA002207010001',
 'K131332206290039',
 'K172122206290032',
 'K131132206280059',
 'K120032206280044',
 'K151412206280064',
 'K140132206270087',
 'K170052206270074',
 'K151572206270056',
 'K170052206270048',
 'K160042206270016',
 'K162142206270009',
 'K120142206230051',
 'K120612206230051',
 'K151122206220105',
 'K170052206220007',
 'K151642206210010',
 'K140022206200117',
 'K1600122061

In [77]:
len(guin_num_all) # 3가지 구인정보 개수 156개 

156

#### 상세정보 얻는 url  방식은 위와 같다. guin_num_all 의 0번을 먼저 확인하자.
http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=[인증키]&callTp=D&returnType=XML
&wantedAuthNo=[구인인증번호]&infoSvc=VALIDATION

In [85]:
key = 'WNL596OLQCP7023TCKJPB2VR1HJ'
url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey='+key+\
'&callTp=D&returnType=XML&wantedAuthNo='+guin_num_all[0]+'&infoSvc=VALIDATION'
print(url)

http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=WNL596OLQCP7023TCKJPB2VR1HJ&callTp=D&returnType=XML&wantedAuthNo=K151632207080067&infoSvc=VALIDATION


In [86]:
res = requests.get(url)
text = res.text 
text

'<?xml version="1.0" encoding="UTF-8"?><wantedDtl><wantedAuthNo>K151632207080067</wantedAuthNo><corpInfo><corpNm>주식회사디엠시스템엔지니어링</corpNm><reperNm>김지환</reperNm><totPsncnt>1 명</totPsncnt><capitalAmt>79 백만원</capitalAmt><yrSalesAmt>39 백만원</yrSalesAmt><indTpCdNm>그 외 기타 전자부품 제조업</indTpCdNm><busiCont>스마트비상유도등제조</busiCont><corpAddr>13174 경기도 성남시 중원구 광명로 377, 신구대학 창업관 103호 (금광동)</corpAddr><homePg></homePg><busiSize>중소기업</busiSize></corpInfo><wantedInfo><jobsNm>모바일 애플리케이션 프로그래머(앱·어플 개발)(133207)</jobsNm><wantedTitle>(경력) 데이터베이스 개발 및 어플리케이션 유지관리 및 개발</wantedTitle><relJobsNm>데이터베이스 운영·관리자 시스템 소프트웨어 개발자</relJobsNm><jobCont>저희 회사는 디엠시스템엔지니어링으로 &#xd;\n스마트레이저피난기를 개발하여 제조하는 업체 입니다.&#xd;\n&#xd;\n직무 &#xd;\n데이터베이스 개발관리 및 유지보수&#xd;\n어플이케이션 개발관리 및 유지보수&#xd;\n&#xd;\nAWS  데이터베이스 사용&#xd;\nPython 활용자 &#xd;\nc# 언어 활용자</jobCont><receiptCloseDt>채용시까지</receiptCloseDt><empTpNm>기간의 정함이 없는 근로계약(시간(선택)제)/ 파견근로 비희망/ 대체인력채용 비희망</empTpNm><collectPsncnt>1</collectPsncnt><salTpNm>월급2,500,000원 이상 ~ 2,500,000원 이하,</salTpNm><ent

In [87]:
soup = BeautifulSoup(text,'lxml-xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<wantedDtl><wantedAuthNo>K151632207080067</wantedAuthNo><corpInfo><corpNm>주식회사디엠시스템엔지니어링</corpNm><reperNm>김지환</reperNm><totPsncnt>1 명</totPsncnt><capitalAmt>79 백만원</capitalAmt><yrSalesAmt>39 백만원</yrSalesAmt><indTpCdNm>그 외 기타 전자부품 제조업</indTpCdNm><busiCont>스마트비상유도등제조</busiCont><corpAddr>13174 경기도 성남시 중원구 광명로 377, 신구대학 창업관 103호 (금광동)</corpAddr><homePg/><busiSize>중소기업</busiSize></corpInfo><wantedInfo><jobsNm>모바일 애플리케이션 프로그래머(앱·어플 개발)(133207)</jobsNm><wantedTitle>(경력) 데이터베이스 개발 및 어플리케이션 유지관리 및 개발</wantedTitle><relJobsNm>데이터베이스 운영·관리자 시스템 소프트웨어 개발자</relJobsNm><jobCont>저희 회사는 디엠시스템엔지니어링으로 
스마트레이저피난기를 개발하여 제조하는 업체 입니다.

직무 
데이터베이스 개발관리 및 유지보수
어플이케이션 개발관리 및 유지보수

AWS  데이터베이스 사용
Python 활용자 
c# 언어 활용자</jobCont><receiptCloseDt>채용시까지</receiptCloseDt><empTpNm>기간의 정함이 없는 근로계약(시간(선택)제)/ 파견근로 비희망/ 대체인력채용 비희망</empTpNm><collectPsncnt>1</collectPsncnt><salTpNm>월급2,500,000원 이상 ~ 2,500,000원 이하,</salTpNm><enterTpNm>경력 (최소3년) 필수</enterTpNm><eduNm>학력무관</eduNm><forLang/><m

In [98]:
# 내가 얻어야할 정보들만 따로 추출해야한다. 
pd.read_clipboard('><')
# 보기 힘드니 따로 홈페이지 상세정보에서 따로 정리하겠다. 

,항목\t\t\t타입\t설명
0,<wantedDtl>
1,<wantedAuthNo>\tString\t구인인증번호\t</wantedAuthNo>
2,<corpInfo>
3,<corpNm>\tString\t회사명\t</corpNm>
4,<reperNm>\tString\t대표자명\t</reperNm>
...,...
113,<empChargerDpt>\tString\t채용부서\t</empChargerDpt>
114,<contactTelno>\tString\t전화번호\t</contactTelno>
115,<chargerFaxNo>\tString\t팩스번호\t</chargerFaxNo>
116,</empchargeInfo>


## 구인정보 상세   필요한것만 넘버링
1. <corpNm>	String	회사명	</corpNm>
<reperNm>	String	대표자명	</reperNm>
2. <totPsncnt>	Number	근로자수	</totPsncnt>
3. <capitalAmt>	Number	자본금	</capitalAmt>
4. <yrSalesAmt>	Number	연매출액	</yrSalesAmt>
5. <indTpCdNm>	String	업종	</indTpCdNm>
6. <busiCont>	String	주요사업내용	</busiCont>
7. <corpAddr>	String	회사주소	</corpAddr>
<homePg>	String	회사홈페이지	</homePg>
8. <busiSize>	String	회사규모	</busiSize>
9. <jobsNm>	String	모집집종	</jobsNm>
10. <wantedTitle>	String	구인제목	</wantedTitle>
<relJobsNm>	String	관련직종	</relJobsNm>
11. <jobCont>	String	직무내용	</jobCont>
12. <receiptCloseDt>	String	접수마감일	</receiptCloseDt>
13. <empTpNm>	String	고용형태	</empTpNm>
14. <collectPsncnt>	String	모집인원	</collectPsncnt>
15. <salTpNm>	String	임금조건	</salTpNm>
16. <enterTpNm>	String	경력조건	</enterTpNm>
17. <eduNm>	String	학력	</eduNm>
18. <forLang>	String	외국어	</forLang>
19. <major>	String	전공	</major>
20. <certificate>	String	자격면허	</certificate>
21. <mltsvcExcHope>	String	병역특례채용희망	</mltsvcExcHope>
22. <compAbl>	String	컴퓨터활용능력	</compAbl>
23. <pfCond>	String	우대조건	</pfCond>
24. <etcPfCond>	String	기타우대조건	</etcPfCond>
25. <selMthd>	String	전형방법	</selMthd>
26. <rcptMthd>	String	접수방법	</rcptMthd>
27. <submitDoc>	String	제출서류준비물	</submitDoc>
28. <etcHopeCont>	String	기타안내	</etcHopeCont>
29. <workRegion>	String	근무예정지	</workRegion>
30. <nearLine>	String	인근전철역	</nearLine>
31. <workdayWorkhrCont>	String	근무시간/형태	</workdayWorkhrCont>
32. <fourIns>	String	연금4대 보험	</fourIns>
33. <retirepay>	String	퇴직금	</retirepay>
34. <etcWelfare>	String	기타복리후생	</etcWelfare>
35. <disableCvntl>	String	장애인편의시설	</disableCvntl>
36. <dtlRecrContUrl>	String	상세모집내용 URL	</dtlRecrContUrl>
 <minEdubgIcd>	String	최소학력코드 </minEdubgIcd>
<regionCd>	String	근무지역코드	</regionCd>
<empTpCd>	String	고용형태코드  </empTpCd>
<enterTpCd>	String	근무형태코드  </enterTpCd>
<salTpCd>	String	임금형태코드</salTpCd>
<staAreaRegionCd>	String	근무지지하철지역코드	</staAreaRegionCd>
<walkDistCd>	String	근무지지하철출구거리 </walkDistCd>
37. <contactTelno>	String	전화번호	</contactTelno>





In [99]:
info = ['corpNm','totPsncnt','capitalAmt','yrSalesAmt','indTpCdNm','busiCont',
       'corpAddr','busiSize','jobsNm','wantedTitle','jobCont','receiptCloseDt',
       'empTpNm','collectPsncnt','salTpNm','enterTpNm','eduNm','forLang',
       'major','certificate','mltsvcExcHope','compAbl','pfCond','etcPfCond',
       'selMthd','rcptMthd','submitDoc','etcHopeCont','workRegion','nearLine',
       'workdayWorkhrCont','fourIns','retirepay','etcWelfare','disableCvntl',
       'dtlRecrContUrl','contactTelno']
info

['corpNm',
 'totPsncnt',
 'capitalAmt',
 'yrSalesAmt',
 'indTpCdNm',
 'busiCont',
 'corpAddr',
 'busiSize',
 'jobsNm',
 'wantedTitle',
 'jobCont',
 'receiptCloseDt',
 'empTpNm',
 'collectPsncnt',
 'salTpNm',
 'enterTpNm',
 'eduNm',
 'forLang',
 'major',
 'certificate',
 'mltsvcExcHope',
 'compAbl',
 'pfCond',
 'etcPfCond',
 'selMthd',
 'rcptMthd',
 'submitDoc',
 'etcHopeCont',
 'workRegion',
 'nearLine',
 'workdayWorkhrCont',
 'fourIns',
 'retirepay',
 'etcWelfare',
 'disableCvntl',
 'dtlRecrContUrl',
 'contactTelno']

In [110]:
soup.find('corpNm').text

'주식회사디엠시스템엔지니어링'

In [103]:
soup.find(info[5]).text

'스마트비상유도등제조'

In [148]:
worknet_df = pd.DataFrame(columns=['회사명','근로자수','자본금','연매출액',
                                  '업종','주요사업내용','회사주소','회사규모',
                                  '모집직종','구인제목','직무내용','접수마감일','고용형태',
                                  '모집인원','임금조건','경력조건','학력','외국어','전공',
                                  '자격면허','병역특례채용희망','컴활능력','우대조건','기타우대조건',
                                  '전형방법','접수방법','제출서류준비물','기타안내','근무예정지',
                                  '인근전철역','근무시간/형태','연금4대보험','퇴직금','기타복리후생','장애인편의시설',
                                  '상세모집내용URL','전화번호'])

worknet_df

,회사명,근로자수,자본금,연매출액,업종,주요사업내용,회사주소,회사규모,모집직종,구인제목,...,기타안내,근무예정지,인근전철역,근무시간/형태,연금4대보험,퇴직금,기타복리후생,장애인편의시설,상세모집내용URL,전화번호


In [149]:
# 상세 정보에 없는 값이 있어서 AttributeError 발생 해서 예외처리 작업 실시 , 오류 발생 시 nan 값을 집어넣었다.
key = 'WNL596OLQCP7023TCKJPB2VR1HJ'
for i in guin_num_all:
    url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey='+key+\
    '&callTp=D&returnType=XML&wantedAuthNo='+i+'&infoSvc=VALIDATION'
    res = requests.get(url)
    text = res.text
    soup = BeautifulSoup(text,'lxml-xml')
    info_list = []
    for j in info:
        try:        
            info_list.append(soup.find(j).text)
        except AttributeError:
            info_list.append(np.nan)
    worknet_df = worknet_df.append(pd.Series(info_list,index=worknet_df.columns),ignore_index=True)
worknet_df

,회사명,근로자수,자본금,연매출액,업종,주요사업내용,회사주소,회사규모,모집직종,구인제목,...,기타안내,근무예정지,인근전철역,근무시간/형태,연금4대보험,퇴직금,기타복리후생,장애인편의시설,상세모집내용URL,전화번호
0,주식회사디엠시스템엔지니어링,1 명,79 백만원,39 백만원,그 외 기타 전자부품 제조업,스마트비상유도등제조,"13174 경기도 성남시 중원구 광명로 377, 신구대학 창업관 103호 (금광동)",중소기업,모바일 애플리케이션 프로그래머(앱·어플 개발)(133207),(경력) 데이터베이스 개발 및 어플리케이션 유지관리 및 개발,...,,"(13174) 경기도 성남시 중원구 광명로 377, 신구대학 창업관 103호 (금광동)",8호선 단대오거리 3번출구 800M,"평일 : (오전) 10시 00분~(오후) 5시 00분, 주 5일 근무, 평균근무시간...",국민연금 고용보험 산재보험 의료보험,퇴직금,,장애인용 주차장,http://www.work.go.kr/outOffer/empInfo/empDeta...,031-524-9895
1,(주)뉴비전네트웍스,70 명,0 백만원,0 백만원,측량업,"수치지도제작및판매, 소프트웨어개발 및 영상처리, 측지측량","51331 경상남도 창원시 마산합포구 산호남로 48, 3층 (산호동)",중소기업,데이터 전문가(134100),(주)뉴비전네트웍스 T-MAP 컨텐츠그룹(GIS 전자지도구축)신입/경력사원 공개채용,...,,"(06748) 서울특별시 서초구 바우뫼로 178, 6F, 티맵모빌리티 LBS센터 ...",3호선 양재 8번출구 500M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,
2,(재)울산연구원,106 명,100 백만원,3000 백만원,기타 인문 및 사회과학 연구개발업,울산광역시 위탁 연구개발업무 수행,"44720 울산광역시 남구 삼산중로 6, 울산연구원",중소기업,사회과학 연구원(110200),울산연구원 울산 빅데이터센터 전문위원(임시직) 공개채용 공고,...,,"(44720) 울산광역시 남구 삼산중로 6, 울산연구원",,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,해당없음,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,052-283-7733
3,한국섬유개발연구원,90 명,0 백만원,0 백만원,사업지원 서비스업,기술검사서비스,41842 대구광역시 서구 국채보상로 136 (중리동),기타,데이터 전문가(134100),한국섬유개발연구원 연구직 - 계약직 채용,...,,(41842) 대구광역시 서구 국채보상로 136 (중리동),2호선 감삼 4번출구 2KM,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,기타(유류지원비),,http://www.work.go.kr/outOffer/empInfo/empDeta...,053-560-6777
4,주식회사 모든다해,28 명,500 백만원,5000 백만원,그 외 기타 특수목적용 기계 제조업,금속검출기 제조업,14341 경기도 광명시 가학로95번길 13 (가학동),중소기업,데이터 설계 및 프로그래머(134101),"금속검출기, 중량선별기 프로그래밍 연구원 모집 (하드웨어/R/F 펨웨어)",...,,"(14322) 경기도 광명시 하안로 60, 광명SK테크노파크 D동 1110호 (소하동)",1호선 석수 1번출구 2KM,"평일 : (오전) 9시 00분~(오후) 6시 00분 (휴게시간 포함), 주 5일 근...",국민연금 고용보험 산재보험 의료보험,퇴직연금,통근버스,,http://www.work.go.kr/outOffer/empInfo/empDeta...,070-7463-9721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,(주)한솥,35 명,105 백만원,99590 백만원,기타 가공식품 도매업,도시락 체인본부,"135937 서울특별시 강남구 강남대로 318, 타워837빌딩 8,9층 (역삼동)",중견기업,데이터 전문가(134100),(주)한솥 디지털 기획 운영팀 IT담당 경력사원 채용,...,,"(135937) 서울특별시 강남구 강남대로 318, 타워837빌딩 8,9층 (역삼동)",신분당선 강남 4번출구 200M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,"기타(건강검진, 경조휴가, 경조금, 자녀학자금, 휴양소(법인콘도), 독감예방접종, ...",,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-2033-9632
152,나이스지니데이타,56 명,0 백만원,0 백만원,데이터베이스 및 온라인정보 제공업,데이터 분석 및 판매,"07242 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신관) ...",중견기업,데이터 분석가(빅데이터 분석가)(134102),[NICE 지니데이타] 빅데이터분석 컨설팅 신입직원 채용,...,,"(07242) 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신...",9호선 국회의사당 3번출구 200M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,교육비 지원,,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-2639-2398
153,나이스지니데이타,56 명,0 백만원,0 백만원,데이터베이스 및 온라인정보 제공업,데이터 분석 및 판매,"07242 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신관) ...",중견기업,데이터 분석가(빅데이터 분석가)(134102),[NICE 지니데이타] 빅데이터분석 컨설팅 경력직원 채용,...,,"(07242) 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신...",,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-2639-2398
154,(주)지오비전,40 명,202 백만원,799 백만원,응용 소프트웨어 개발 및 공급업,지능형 영상분석 솔루션,"24341 강원도 춘천시 강원대학길 1, 보듬관 404, 405호",중소기업,시스템 소프트웨어 개발자(133100),데이터 분석 및 빅데이터 기반 딥러닝 경력/석사 채용(서울/춘천),...,,"(24341) 강원도 춘천시 강원대학길 1, 보듬관 404, 405호",2호선 구로디지털단지 2번출구 800M,"평일 : (오전) 9시 30분 ~ (오후) 6시 30분, 주 5일 근무, 평균근무시...",국민연금 고용보험 산재보험 의료보험,퇴직금,"기타(- 주 5일 근무 ( 연차,반차 ) -- 주5일근무시간 (탄력출퇴근제)\r\...",,http://www.work.go.kr/outOffer/empInfo/empDeta...,


In [151]:
worknet_df.shape

(156, 37)

In [161]:
worknet_df.to_csv('/Users/snoo/enterprise/worknet_df.csv',index=False)

In [175]:
worknet_df =pd.read_csv('/Users/snoo/enterprise/worknet_df.csv')

In [184]:
worknet_df1 = worknet_df.copy()

In [185]:
# 태블로우에서 nan 값이나 ''값은 인식 못해서 스페이스 하나 넣어줬다. = 태블로우에선 Null 로 나온다. 
# 이제 주소만 따서 좌표값 추가해주고 바꿔줄거 자본금, 매출 등등 숫자형태로 바꿔줘야겠다. 
worknet_df1 = worknet_df1.fillna(' ')
worknet_df1.to_csv('/Users/snoo/enterprise/worknet_df1.csv',index=False)

In [187]:
worknet_df1.loc[2]

회사명                                                   (재)울산연구원
근로자수                                                     106 명
자본금                                                    100 백만원
연매출액                                                  3000 백만원
업종                                          기타 인문 및 사회과학 연구개발업
주요사업내용                                      울산광역시 위탁 연구개발업무 수행
회사주소                             44720 울산광역시 남구 삼산중로 6, 울산연구원 
회사규모                                                      중소기업
모집직종                                          사회과학 연구원(110200)
구인제목                         울산연구원 울산 빅데이터센터 전문위원(임시직) 공개채용 공고
직무내용         울산 빅데이터센터 운영 연구직\r\n∘채용직급:  전문위원\r\n∘채용인원: 1명 ...
접수마감일                                                 20220718
고용형태                 기간의 정함이 있는 근로계약6 개월/ 파견근로 비희망/ 대체인력채용 비희망
모집인원                                                         1
임금조건              월급2,885,720원 이상 ~ 3,649,570원 이하, 면접 후 재조정 가능
경력조건                                                   

#### kakao API 이용 주소를 이용 좌표로 변경 (tableau 맵을 이용해 회사 위치 파악)